## 举例1：大模型分析工具的调用

In [16]:
from langchain_core.utils.function_calling import convert_to_openai_function
from langchain_community.tools import MoveFileTool
from langchain_core.messages import HumanMessage
from langchain_openai import ChatOpenAI
import os
import dotenv
from markdown_it.cli.parse import convert

# 1、加载环境变量
dotenv.load_dotenv()
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY1")
os.environ["OPENAI_BASE_URL"] = os.getenv("OPENAI_BASE_URL")

# 2、获取大模型
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

# 3、获取工具列表
tools = [MoveFileTool()]  # 这里可以添加更多工具
# 4、因为大模型invoke调用时需要传入函数的列表，所以需要把工具转换成函数
function = [convert_to_openai_function(tool) for tool in tools]

# 5、获取消息列表
messages = [HumanMessage(content="将文件a移动到桌面")]

# 6、调用大模型（传入消息、工具列表）
response = llm.invoke(
	input=messages,
	# tools=tools, # 不支持直接传入工具列表
	functions=function,
)
print(response)

content='' additional_kwargs={'function_call': {'arguments': '{"source_path":"a","destination_path":"/Users/YourUsername/Desktop/a"}', 'name': 'move_file'}, 'refusal': None} response_metadata={'token_usage': {'completion_tokens': 27, 'prompt_tokens': 76, 'total_tokens': 103, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_efad92c60b', 'id': 'chatcmpl-COkhUQwXg8lvkmdMRoz3oaozTBOtx', 'service_tier': None, 'finish_reason': 'function_call', 'logprobs': None} id='run--57ccc241-747f-49f5-8c1a-373c5a8c8b01-0' usage_metadata={'input_tokens': 76, 'output_tokens': 27, 'total_tokens': 103, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}}


In [17]:
# 作为对比
from langchain_core.utils.function_calling import convert_to_openai_function
from langchain_community.tools import MoveFileTool
from langchain_core.messages import HumanMessage
from langchain_openai import ChatOpenAI
import os
import dotenv
from markdown_it.cli.parse import convert

# 1、加载环境变量
dotenv.load_dotenv()
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY1")
os.environ["OPENAI_BASE_URL"] = os.getenv("OPENAI_BASE_URL")

# 2、获取大模型
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

# 3、获取工具列表
tools = [
	MoveFileTool(),
]  # 这里可以添加更多工具

# 4、因为大模型invoke调用时需要传入函数的列表，所以需要把工具转换成函数
functions = [convert_to_openai_function(tool) for tool in tools]

# 5、获取消息列表
messages = [HumanMessage(content="查询上海明天的天气信息")]

# 6、调用大模型（传入消息、工具列表）
response = llm.invoke(
	input=messages,
	# tools=tools, # 不支持直接传入工具列表
	functions=functions,
)

print(response)

content='抱歉，我无法提供实时的天气信息。建议您查看天气预报网站或使用天气应用程序获取上海明天的天气信息。' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 32, 'prompt_tokens': 76, 'total_tokens': 108, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_efad92c60b', 'id': 'chatcmpl-COki1GgHcWS0MjZQAGvziYwFpQmLD', 'service_tier': None, 'finish_reason': 'stop', 'logprobs': None} id='run--775c4175-cd30-4e73-93a3-a81df6d1f0b3-0' usage_metadata={'input_tokens': 76, 'output_tokens': 32, 'total_tokens': 108, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}}


通过上述2个例子可以看出，得到的AIMessages的核心属性如下：
- 如果分析出需要调用对应的工具：

	- content：信息为空。因为大模型要调用工具，所以不需要返回内容
	 - additional_kwargs：包含function_call字段，指明具体函数调用的参数和函数名。
- 如果没有分析出需要调用对应的工具：
	- content：信息不为空。包含大模型的回答内容
	 - additional_kwargs：不包含function_call字段。

## 举例2：如何调用具体大模型分析出来的工具

说明：

1. 大模型与Agent的区别：是否涉及到工具的调用
2. 针对于大模型：仅能分析出要调用的工具，但是此工具（或函数）不能真正被调用
3. 针对于Agent：不仅能分析出要调用的工具，而且此工具（或函数）能真正被调用


In [26]:
from langchain_core.utils.function_calling import convert_to_openai_function
from langchain_community.tools import MoveFileTool
from langchain_core.messages import HumanMessage
from langchain_openai import ChatOpenAI
import os
import dotenv
from markdown_it.cli.parse import convert

# 1、加载环境变量
dotenv.load_dotenv()
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY1")
os.environ["OPENAI_BASE_URL"] = os.getenv("OPENAI_BASE_URL")

# 2、获取大模型
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

# 3、获取工具列表
tools = [MoveFileTool()]  # 这里可以添加更多工具
# 4、因为大模型invoke调用时需要传入函数的列表，所以需要把工具转换成函数
function = [convert_to_openai_function(tool) for tool in tools]

# 5、获取消息列表
messages = [HumanMessage(content=r"将当前目录下的文件a.txt移动到C:\Users\Chenxr\Desktop")]

# 6、调用大模型（传入消息、工具列表）
response = llm.invoke(
	input=messages,
	# tools=tools, # 不支持直接传入工具列表
	functions=function,
)
# print(response)

### 步骤一：分析要调用哪个工具或函数

In [29]:
import json

if "function_call" in response.additional_kwargs:
	# 1、获取函数名/工具名
	tool_name = response.additional_kwargs["function_call"]["name"]
	# 2、获取函数参数
	tool_args = json.loads(response.additional_kwargs["function_call"]["arguments"])
	print(f"函数名/工具名：{tool_name}")
	print(f"函数参数：{tool_args}")
else:
	print(f"模型回复：{response.content}")

函数名/工具名：move_file
函数参数：{'source_path': 'a.txt', 'destination_path': 'C:\\Users\\Chenxr\\Desktop\\a.txt'}


### 步骤二：根据函数名/工具名，调用对应的工具或函数

In [30]:
if "move_file" in response.additional_kwargs["function_call"]["name"]:
	tool = MoveFileTool()
	result = tool.run(tool_args)  # 调用工具
	print(f"工具调用结果：{result}")

工具调用结果：File moved successfully from a.txt to C:\Users\Chenxr\Desktop\a.txt.
